# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](https://github.com/RamziRBM/lab-neural-networks/blob/master/your-code/tttboard.jpg?raw=1)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [16]:
import pandas as pd
import numpy as np


df_file = pd.read_csv("/content/tic-tac-toe.csv")

print("=== Training Data Preview ===")
print(df_file.head(), "\n")
print("=== Training Data Info ===")
print(df_file.info(), "\n")
print("=== Summary Statistics ===")
print(df_file.describe(), "\n")

df_file.dtypes

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
encoder = LabelEncoder()

# Apply it to every column
df_encoded = df_file.apply(encoder.fit_transform)


X = df_encoded.iloc[:, :-1]  # All columns except last
y = df_encoded.iloc[:, -1]   # Last column only

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)



=== Training Data Preview ===
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True 

=== Training Data Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None 

=== Summary Statistics ===
         TL   TM   TR   ML   MM   MR   BL   BM   BR class
count   958  958  958  958  958  958  958  958

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [86]:
from __future__ import print_function

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop


from keras.optimizers import Adam


##### Build the model

# Lets build a sequencial model
# model = Sequential([
#    LAYER1 -> Dense(                       -> Dense networks means all neurons in one layer are connected to all neuronsof the next layer
#                    64,                    -> Number of neurons
#                    activation='relu',     -> activation function
#                    input_shape=(784,)),   -> input shape (as it is the first layer)
#                               # 784= 28*28
#
#    LAYER 2 -> Dense(64, activation='relu'),      # choosing relu instead of sigmoid, this is somewhat common
#
#    LAYER 3 (output) -> Dense(
#                              1,           -> # of Output Neurons = # of Classes
#                              activation='softmax'),   # Softmax -> Coverts a universe of real numbers into probabilities!!
#                               ])

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(9,)))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Activation("relu"))
model.add(Dense(32, activation='relu'))
model.add(Activation("relu"))
model.add(Dense(32, activation='relu'))
model.add(Activation("relu"))
model.add(Dense(16, activation='relu'))
model.add(Activation("relu"))
model.add(Dense(16, activation='relu'))
model.add(Activation("relu"))


model.add(Dense(2, activation='softmax'))


model.summary()

my_opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001, epsilon=0.1, decay=0.0)

model.compile(loss='sparse_categorical_crossentropy',
#              optimizer='adam', metrics=['accuracy'])
              optimizer=my_opt, metrics=['accuracy'])

# Define the optimizer
#my_opt = Adam(learning_rate=0.001)
#my_opt = tensorflow.keras.optimizers.Adagrad(learning_rate=0.01, epsilon=0.1, decay=0.0)

#model.compile(loss='sparse_categorical_crossentropy',
#              optimizer = adam,
#              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=10,
                    epochs=15,
                    verbose=1,
                    validation_data=(X_test, Y_test))


print("=== ***************** ===")

score = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Model: "sequential_78"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_471 (Dense)               │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_391 (Activation)     │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_472 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_392 (Activation)     │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_473 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_393 (Activation)     │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_474 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_394 (Activation)     │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_475 (Dense)               │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_395 (Activation)     │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_476 (Dense)               │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_396 (Activation)     │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_477 (Dense)               │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,506 (52.76 KB)

 Trainable params: 13,506 (52.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4151 - loss: 0.6980 - val_accuracy: 0.6198 - val_loss: 0.6895
Epoch 2/15
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5514 - loss: 0.6897 - val_accuracy: 0.6510 - val_loss: 0.6802
Epoch 3/15
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6643 - loss: 0.6769 - val_accuracy: 0.6510 - val_loss: 0.6718
Epoch 4/15
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6588 - loss: 0.6690 - val_accuracy: 0.6510 - val_loss: 0.6654
Epoch 5/15
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6400 - loss: 0.6659 - val_accuracy: 0.6510 - val_loss: 0.6593
Epoch 6/15
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6341 - loss: 0.6618 - val_accuracy: 0.6510 - val_loss: 0.6545
Epoch 7/15
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6573 - loss: 0.6496 - val_accuracy: 0.6510 - val_loss: 0.6511
Epoch 8/15
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6692 - loss: 0.6403 - val_accuracy: 0.6510 - val_loss:

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here